Required files:  
creds.txt: SAM.gov API key  
naics.txt: Newline-separated list of NAICS codes  

TODOS:
* Ask if "Close date" should be Original Date Offers Due or Original Inactive Date
* URGENT: adding too many NAICS Codes pushes the rest of the UI off the edge making it unusable
* general: create some user docs
* make window more resize-friendly and generally less fragile
* improve error reporting UX
* add icon and custom naming to build command
* add development flow for using pre-downloaded data rather than requests
* confirm whether they'll have access to more than 10 requests / day
* figure out how you're going to package and ship this over to them (just a .zip?), antivirus problems?
* allow right-click copy-and-paste if possible
* try and reduce the size of the final program considerably

In [8]:
# import libraries
from requests import get
from json import loads
import datetime
import PySimpleGUI as sg
from pandas import DataFrame, concat
from time import sleep

In [9]:
# get API key
def fetch_api_key():
    with open("./creds.txt") as creds_file:
        contents = creds_file.read().replace(" ", "").replace("\n", "")
        if (not contents):
            raise FileNotFoundError("There's something wrong with your API key in creds.txt")
        return contents
    

In [10]:
def generate_request(limit, naics_code, api_key):
    base_uri = "https://api.sam.gov/prod/opportunities/v2/search"
    limit_string = "?limit=" + str(1000)

    # limit date to today mm/dd/yyyy
    now = datetime.datetime.now().date()
    now_date_string = str(now.month) + "/" + str(now.day) + "/" + str(now.year)
    date_range_string = "&postedFrom=" + now_date_string + "&postedTo=" + now_date_string

    # notice type: Combined Synopsis / Solicitation
    ptype_string = "&ptype=k"

    # NAICS codes
    ncode_string = "&ncode=" + naics_code

    # set aside: Total Small Business (19.5)
    set_aside_string = "&typeOfSetAside=SBA"

    # status: active
    # TODO (GSA haven't implemented somehow as of 10/7/2023)

    request = base_uri + limit_string + "&api_key=" + api_key + date_range_string + ptype_string + set_aside_string + ncode_string
    return request

In [11]:
# fields I care about:
# * Portal: static value for now "Sam.gov"
# * Date: today's date
# * NAICS Code: part of the search, but also part of the response
# * SolicitationNumber: string (call the column "RFQ/Solicitation/Buy #")
# * RFQ TITLE: title: string
# * ISSUE DATE: postedDate: date as string
# * Close Date: responseDeadLine: date as string

def create_opportunity_record(opportunity):
    record = {}
    record["Portal"] = ["SAM.gov"]
    record["DATE"] = [datetime.datetime.now().ctime()]
    record["NAICS Code"] = [opportunity["naicsCode"]]
    record["RFQ/Solicitation/Buy #"] = [opportunity["solicitationNumber"]]
    record["RFQ TITLE"] = [opportunity["title"]]
    record["ISSUE DATE"] = [opportunity["postedDate"]]
    record["Close Date"] = [opportunity["responseDeadLine"]]
    return DataFrame.from_dict(record)

In [12]:
# goes through the provided text file of naics
def get_selected_naics():
    naics = []
    with open("./naics.txt") as naics_file:
        for line in naics_file:
            line = line.replace("\n", "")
            naics.append(line)
    return naics

## UI

In [13]:
# custom components
def double_button(text):
    return [sg.pin(sg.Button(text, pad=((5,0),(5,5)), disabled=True, disabled_button_color=("black", None), k="button_"+text)), sg.pin(sg.Button("X", pad=((0,5),(5,5)), k="remove_"+text))]

def current_naics_layout(naics_list):
    return [double_button(naics_code) for naics_code in naics_list]

In [14]:
# ENV SETUP
api_key = fetch_api_key()
df = DataFrame(columns=["Portal", "DATE", "NAICS Code", "RFQ/Solicitation/Buy #", "RFQ TITLE", "ISSUE DATE", "Close Date"])


# UI SETUP
sg.theme('DarkBlue2') 

naics_list = get_selected_naics()
current_naics_list = naics_list.copy()

layout = [  [sg.Text('Welcome to the SAM.gov Query Tool. \nPress the \"SEND REQUESTS\" button to request today\'s contract opportunities \nfor the selected NAICS codes.', size=(80,10))],
            [sg.Frame('Enter additional NAICS codes:', [[sg.Combo(values=naics_list, key="new_naics", enable_events=True, bind_return_key=True)]])],
            [sg.Frame('Currently selected NAICS codes:', layout=current_naics_layout(naics_list), k="current_naics")],
            [sg.Button('SEND REQUESTS', k="send_requests")],
            [sg.Output(size=(80, 10))] ]


window = sg.Window('SAM.gov Query Tool', layout, size=(600,500))

while True:
    event, values = window.read()
    # print("event: " + str(event))
    # print("values: ")
    # print(values)

    if event == sg.WIN_CLOSED or event == 'Cancel': # if user closes window or clicks cancel
        break

    elif event == "new_naics":
        new_naic = values[event]
        if (not new_naic in current_naics_list):
            current_naics_list.append(new_naic)
            naics_list.append(new_naic)
            window['new_naics'].update(values=naics_list)
            window.extend_layout(window['current_naics'], [double_button(new_naic)])

    elif event.startswith("remove_"):
        # get the number from after prefix
        naic = event[event.index("_") + 1:]
        current_naics_list.remove(naic)
        window[event].update(visible=False)
        window["button_"+naic].update(visible=False)

    elif event == "send_requests":
        # for each naics code:
        # parse response object
        # grab fields you care about and cram em into a CSV
        for naics_code in current_naics_list:
            print("Attempting to get contract opportunities for NAICS code " + naics_code + "...")
            response = get(generate_request(10, naics_code, api_key))
            print("Response status code: " + str(response.status_code))
            if (str(response.status_code).startswith("2")):
                content = loads(response.content)
                print("Successful response. Found " + str(content["totalRecords"]) + " records.")
                print("Creating records...")
                for opportunity in content["opportunitiesData"]:
                    record = create_opportunity_record(opportunity)
                    print("Adding record " + str(record))
                    df = concat([df, record])
            elif (str(response.status_code) == "429"):
                print("This api key is being rate limited. Please try again later. For more information on rate limits, see: https://www.fsd.gov/gsafsd_sp?id=gsafsd_kb_articles&sys_id=ee6c594b1bdb251006b09796bc4bcb4a")
            sleep(1)
        # output the df as a CSV
        filename = "Contract_Opportunities_" + datetime.datetime.now().ctime().replace(":", "-").replace(" ", "_") + ".xlsx"
        df.to_excel(filename, index=False)
        print("Created file " + filename)
        
            


window.close()

In [15]:
# raw_json = ""
# with open("test_data.txt") as file:
#     raw_json = file.read()
# obj = json.loads(raw_json)

In [16]:
# rec = create_opportunity_record(obj["opportunitiesData"][0])

In [17]:
# df = DataFrame(columns=["Portal", "DATE", "NAICS Code", "RFQ/Solicitation/Buy #", "RFQ TITLE", "ISSUE DATE", "Close Date"])

In [18]:
# df = concat([df, DataFrame.from_dict(rec)])

In [19]:
# df

In [20]:
# df.to_excel("test_sheet.xlsx", index=False)